In [1]:
import numpy as np
import keras

Using Theano backend.


In [9]:
x= "0123456789"*100000

print len(x)

1000000


In [11]:
print set(x)
print list(set(x))

set(['1', '0', '3', '2', '5', '4', '7', '6', '9', '8'])
['1', '0', '3', '2', '5', '4', '7', '6', '9', '8']


In [24]:
dct = ['*'] +list(set(x))
max_features = len(dct)

rev_dct= [(j,i) for i,j in enumerate(dct)]
rev_dct= dict(rev_dct)

In [25]:
print dct
print rev_dct

['*', '1', '0', '3', '2', '5', '4', '7', '6', '9', '8']
{'*': 0, '1': 1, '0': 2, '3': 3, '2': 4, '5': 5, '4': 6, '7': 7, '6': 8, '9': 9, '8': 10}


In [26]:
x= [rev_dct[ch] for ch in x]

In [27]:
n_timestamps=200
x= x[:len(x) - len(x) % n_timestamps]
x= np.array(x, dtype='int32').reshape((-1,n_timestamps))

In [37]:
print x.shape
print x[0][:20]

(5000, 200)
[ 2  1  4  3  6  5  8  7 10  9  2  1  4  3  6  5  8  7 10  9]


In [31]:
y = np.zeros((x.shape[0], x.shape[1], max_features), dtype='int32')
for i in np.arange(x.shape[0]):
    for j in np.arange(x.shape[1]):
        y[i, j, x[i, j]] = 1

In [38]:
print y.shape

(5000, 200, 11)


In [39]:
x = np.roll(y, 1, axis=1)
x[:, 0, :] = 0
x[:, 0, 0] = 1

In [44]:
print x.shape
print x[1][2]

(5000, 200, 11)
[0 1 0 0 0 0 0 0 0 0 0]


In [45]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM

model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(200,  max_features)))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256, return_sequences=True))
model.add(Dense(max_features))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_1 (LSTM)                    (None, 200, 256)      274432      lstm_input_1[0][0]               
____________________________________________________________________________________________________
lstm_2 (LSTM)                    (None, 200, 256)      525312      lstm_1[0][0]                     
____________________________________________________________________________________________________
lstm_3 (LSTM)                    (None, 200, 256)      525312      lstm_2[0][0]                     
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 200, 11)       2827        lstm_3[0][0]                     
___________________________________________________________________________________________

In [46]:
hist = model.fit(x, y, batch_size=64, nb_epoch=1, verbose=2)

KeyboardInterrupt: 

In [ ]:
import numpy
def mnrnd(probs):
    rnd = numpy.random.random()
    for i in xrange(len(probs)):
        rnd -= probs[i]
        if rnd <= 0:
            return i
    return i

sentences = numpy.zeros((128, n_timestamps, max_features))
sentences[:, 0, 0] = 1

# Start sampling char-sequences. At each iteration i the probability over
# the i-th character of each sequences is computed. 
for i in numpy.arange(10):
    probs = model.predict_proba(sentences, verbose=2)[:,i,:]
    # Go over each sequence and sample the i-th character.
    for j in numpy.arange(len(sentences)):
        sentences[j, i+1, mnrnd(probs[j, :])] = 1
sentences = [sentence[1:].nonzero()[1] for sentence in sentences]

# Convert to readable text.
text = []
for sentence in sentences:
    text.append(''.join([dct[word] for word in sentence]))
print text

